First, we import the needed libraries.

In [2]:
import tensorflow as tf
import numpy as np
import larq

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, InputLayer, Flatten, Input
from tensorflow.keras.utils import to_categorical



Let us first load the MNIST dataset. The MNIST database, an extension of the NIST database, is a low-complexity data collection of handwritten digits used to train and test various supervised machine learning algorithms. The database contains 70,000 28x28 black and white images representing the digits zero through nine. The data is split into two subsets, with 60,000 images belonging to the training set and 10,000 images belonging to the testing set. The separation of images ensures that given what an adequately trained model has learned previously, it can accurately classify relevant images not previously examined

We use utils.to_categorical to classify the images in the 10 categorical labels, 

In [3]:
#Load data.
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)


In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 10)
(10000, 28, 28)
(10000, 10)


We add a dimension to training data because Keras layers expect to receive a 4-dimension tensor.

In [4]:
x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test,axis=-1)

2022-07-24 17:53:45.987631: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 376320000 exceeds 10% of free system memory.


https://www.sciencedirect.com/science/article/pii/S1352231097004470?casa_token=yCSinRcM2_EAAAAA:9gyr-B6TzJMX946qREaBG97v8T-4VZnSH6CPxmvQmX6DuNnOKi651PwTGyYwj4JIsbEbpX0AJg


We build a MLP with two hidden layers and a ReLU activation function.
Next, we would create a Sequential model and add Dense layers with ‘ReLU’ activation function.

In [5]:
inputs=Input(shape=(28,28,1),name="input_layer")
layer=Flatten()(inputs)
layer=Dense(512, activation='relu',name="hidden1")(layer)
layer=Dense(768, activation='relu',name="hidden2")(layer)
layer=Dense(10, activation='softmax',name="output_layer")(layer)
model = Model(inputs=inputs, outputs=layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 28, 28, 1)]       0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 hidden1 (Dense)             (None, 512)               401920    
                                                                 
 hidden2 (Dense)             (None, 768)               393984    
                                                                 
 output_layer (Dense)        (None, 10)                7690      
                                                                 
Total params: 803,594
Trainable params: 803,594
Non-trainable params: 0
_________________________________________________________________


Finally, we would compile the model with optimizer as adam,  loss as categorical cross-entropy, and 100 number of epochs.
Once trained, we save it.

In [6]:
print(x_train.shape)
print(y_train.shape)
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=64)
model.save("mnist_mlp_2h.h5")

(60000, 28, 28, 1)
(60000, 10)
Epoch 1/100
938/938 [==============================] - 3s 3ms/step - loss: 0.1920 - accuracy: 0.9422
Epoch 2/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0766 - accuracy: 0.9760
Epoch 3/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0530 - accuracy: 0.9830
Epoch 4/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0374 - accuracy: 0.9882
Epoch 5/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0325 - accuracy: 0.9894
Epoch 6/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0267 - accuracy: 0.9912
Epoch 7/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0225 - accuracy: 0.9931
Epoch 8/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0186 - accuracy: 0.9940
Epoch 9/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0204 - accuracy: 0.9934
Epoch 10/100
938/938 [==============================] - 3s 3

We load the configuration for LARQ layers

In [7]:
import json

# Load configuration.
larq_configuration={"pad_values":0.0,
"input_quantizer":None,
"depthwise_quantizer":None,
"pointwise_quantizer":None,
"kernel_quantizer":None}

with open("../configuration/config.json") as json_data_file:
    larq_configuration = json.load(json_data_file)

Now we create a BNN based on the previous model, train it and save the resulting model.

In [8]:
from dnn2bnn.models.model_manager import ModelManager

mm=ModelManager(original_model=model,larq_configuration=larq_configuration)
larq_model=mm.create_larq_model(original_model=model)
larq_model.summary()

#The loaded model has been compiled, but it can be re-compiled with other parameters if wished.
#model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

larq_model.fit(x_train, y_train, epochs=100, batch_size=64)
larq_model.save("bin_mnist_2h.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 quant_dense (QuantDense)    (None, 512)               401920    
                                                                 
 quant_dense_1 (QuantDense)  (None, 768)               393984    
                                                                 
 quant_dense_2 (QuantDense)  (None, 10)                7690      
                                                                 
Total params: 803,594
Trainable params: 803,594
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
938/938 [==============================] - 6s 6ms/step - loss: 1.6451 - accuracy: 0.5146
Epoch 2/100
938/938 [==============================] - 5s 6ms/step - loss: 1.1165

We test the obtained models and compare its fidelity:

In [9]:
#ORIGINAL MLP
score = model.evaluate(x_test, y_test, verbose=0)
print("MLP: Test loss:", score[0])
print("MLP: Test accuracy:", score[1])

MLP: Test loss: 0.3397727906703949
MLP: Test accuracy: 0.982200026512146


In [10]:
#BNN
score = larq_model.evaluate(x_test, y_test, verbose=0)
print("BNN: Test loss:", score[0])
print("BNN Test accuracy:", score[1])

BNN: Test loss: 0.4318749010562897
BNN Test accuracy: 0.8747000098228455


In [8]:
#External Fidelity
import tensorflow as tf
import larq
from dnn2bnn.metrics.fidelity import Fidelity

model=tf.keras.models.load_model("mnist_mlp_2h.h5")
larq_model=tf.keras.models.load_model("bin_mnist_2h.h5")

fidelity=Fidelity(original=model, surrogate=larq_model,x=x_test)
fid_acc=fidelity.accuracy()
print("FIDELITY type=ACCURACY")
print("                 value=" + str(fid_acc))

313/313 [==============================] - 1s 2ms/step
FIDELITY type=ACCURACY
                 value=0.8746
